# Method 

My goal is to do a brute force search for pleasant musical notes to identify where the boundaries of pleasant and unpleasant exists. 

The basic method is to make 2 note chords, one with the fundamental $f_1$ and the other with frequency $f_2 = rf_1$ for some ratio $r$. I test ratios $r = 1 + k * 0.01$ for $r \in [1,2]$. 

* Experiment 1 -- I test this with sine waves 
    - Pros: Simple plots 
    - Cons: unclear if single sine wave can work given that it doesn't have enough density on the harmonics 
* Experiment 2 
    - Pros: better density on the harmonics
    - Cons: complicated plots 

# Experiment 1: Sine waves 

In [ ]:
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from IPython.display import Audio, display

# Parameters
duration = 1.0   # seconds (for audio)
sr = 44100       # sample rate
base_freq = 403.0
t = np.linspace(0, duration, int(sr * duration), endpoint=False)

# Frequency multipliers: 1.005 → 2.005 in EXACT 0.01 steps (101 values)
multipliers = np.round(np.linspace(1.00, 2.00, 101), 2)

window_ms = 12   # how much of waveform to show in plots
print(f"Generating {len(multipliers)} sine chords...")

for idx, m in enumerate(multipliers, start=1):
    f1 = base_freq
    f2 = base_freq * m

    # Individual components
    y1 = np.sin(2 * np.pi * f1 * t).astype(np.float32)
    y2 = np.sin(2 * np.pi * f2 * t).astype(np.float32)

    # Sum and normalize
    y_sum = (y1 + y2).astype(np.float32)
    peak = np.max(np.abs(y_sum))
    if peak > 0:
        y_sum /= peak

    # ---- Save to file (only the sum) ----
    filename = f"sinechord_{int(f1)}Hz_{round(f2, 2)}Hz.wav"
    sf.write(filename, y_sum, sr)

    # ---- Plot (zoomed window) ----
    win_samples = int((window_ms / 1000.0) * sr)
    tt = t[:win_samples]
    plt.figure(figsize=(9, 3.5))
    plt.plot(tt, y1[:win_samples], label="sin(2π c₁ x)")
    plt.plot(tt, y2[:win_samples], label="sin(2π c₂ c₁ x)")
    plt.plot(tt, y_sum[:win_samples], label="sum (normalized)")
    plt.title(f"Chord {idx}: {f1:.2f} Hz + {f2:.2f} Hz (Multiplier: {m:.3f})")
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.legend(loc="upper right")
    plt.tight_layout()
    plt.show()

    # ---- Playback (only the sum) ----
    print(f"Chord {idx}: {f1:.2f} Hz + {f2:.2f} Hz (Multiplier: {m:.3f})")
    display(Audio(y_sum, rate=sr))

print(f"All {len(multipliers)} sine chords generated and saved.")


# Experiment 2: Violins 